In [1]:
from io import open
import glob
import os
import unicodedata
import string

In [2]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets

from torchvision.transforms import ToTensor

In [25]:
from torch import nn
import random

from torch.functional import F

import numpy as np

In [11]:
from torch.utils.data import DataLoader
from torch import optim
import math

In [12]:

def split_to_names(fname):
    EOS = "<EOS>"
    data = []
        
    text = open(fname).read().lower()
            
    names = text.splitlines()
    for i, name in enumerate(names): 
        ch_list = list(name) + [EOS]
        data.append(ch_list)
    return data
names = split_to_names(r"C:\Users\Trijal Srivastava\OneDrive\Desktop\VS CODE\Beyond the buzz\RNN\dinos (1).txt")


In [13]:

char_vocab = ["<EOS>"] + sorted([ch for ch in string.ascii_lowercase])
char_to_ix = {ch:i for i,ch in enumerate(char_vocab)}
ix_to_char = {i:ch for ch,i in char_to_ix.items()}


In [14]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_as_str, _map):
        self.data_as_int = []
        
        # Convert characters to integers
        for seq_as_str in data_as_str:
            seq_as_int = keys_to_values(seq_as_str, _map,
                random.choice(list(_map)))
            
            self.data_as_int.append(seq_as_int)

    def __len__(self):
        return len(self.data_as_int)

    def __getitem__(self, ix):
        # Get data sample at index ix
        item = self.data_as_int[ix]
        
        # Slice x and y from sample
        x = item[:-1]
        y = item[ 1:]
        return torch.tensor(x), torch.tensor(y)

def keys_to_values(keys, _map, default):
    return [_map.get(key, default) for key in keys]

In [15]:
dataset = Dataset(names, char_to_ix)
dataloader = DataLoader(dataset, 1, True)

In [16]:
class Model(nn.Module):
    def __init__(self, _map, hidden_size, emb_dim=8, n_layers=1, dropout_p=0.2):
        super(Model, self).__init__()
        
        self.vocab_size  = len(_map)
        self.hidden_size = hidden_size
        self.emb_dim     = emb_dim
        self.n_layers    = n_layers
        self.dropout_p   = dropout_p
        
        self.embedding = nn.Embedding(
            num_embeddings=self.vocab_size,
            embedding_dim =self.emb_dim)
        
        self.lstm = nn.LSTM(
            input_size =self.emb_dim,
            hidden_size=self.hidden_size,
            num_layers =self.n_layers,
            batch_first=True)
        
        self.dropout = nn.Dropout(self.dropout_p)
        
        self.fc = nn.Linear(
            in_features =self.hidden_size,
            out_features=self.vocab_size)
        
    def forward(self, x, prev_state):
        n_b, n_s = x.shape
        
        embed = self.embedding(x)
        yhat, state = self.lstm(embed, prev_state)
        
        yhat = self.dropout(yhat)
        out = self.fc(yhat)
        return out, state
    
    def init_state(self, b_size=1):
        return (torch.zeros(self.n_layers, b_size, self.hidden_size),
                torch.zeros(self.n_layers, b_size, self.hidden_size))

model = Model(char_to_ix, 64, 8, n_layers=1, dropout_p=0.2)

In [18]:
def train(model, data, num_iter, criterion, clip=0.25, lr=0.001, print_every=50):
    model.train()
    
    costs = []
    running_loss = 0
    optimizer = optim.Adam(model.parameters(), lr=lr)

    curr_iter = 0
    while curr_iter<num_iter:
        for x, y in data:
            optimizer.zero_grad()
            
            # Initialise model's state and perform forward-prop
            prev_state = model.init_state(b_size=x.shape[0])
            out, state = model(x, prev_state)

            # Calculate loss
            loss = criterion(out.transpose(1, 2), y)
            costs.append(loss.item())
            running_loss += loss.item()

            # Calculate gradients and update parameters
            loss.backward()
            if clip:
                nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
            
            curr_iter += 1
            if print_every and (curr_iter%print_every)==0:
                print("Iteration: {:{}}/{}, Loss: {:8.4f}".format(
                    curr_iter, int(math.log(num_iter, 10))+2, num_iter,
                    running_loss/float(print_every)))
                running_loss = 0
                
            if curr_iter>=num_iter:
                break
    return model, costs


criterion = nn.CrossEntropyLoss()
model, costs = train(
    model, dataloader , 50000, criterion, clip=0.25, lr=1e-3, print_every=1000)

Iteration:   1000/50000, Loss:   2.4220
Iteration:   2000/50000, Loss:   2.0090
Iteration:   3000/50000, Loss:   1.8742
Iteration:   4000/50000, Loss:   1.7780
Iteration:   5000/50000, Loss:   1.7585
Iteration:   6000/50000, Loss:   1.7220
Iteration:   7000/50000, Loss:   1.6466
Iteration:   8000/50000, Loss:   1.6684
Iteration:   9000/50000, Loss:   1.6105
Iteration:  10000/50000, Loss:   1.5923
Iteration:  11000/50000, Loss:   1.5850
Iteration:  12000/50000, Loss:   1.5743
Iteration:  13000/50000, Loss:   1.5263
Iteration:  14000/50000, Loss:   1.5551
Iteration:  15000/50000, Loss:   1.5016
Iteration:  16000/50000, Loss:   1.5044
Iteration:  17000/50000, Loss:   1.4913
Iteration:  18000/50000, Loss:   1.4513
Iteration:  19000/50000, Loss:   1.4622
Iteration:  20000/50000, Loss:   1.4577
Iteration:  21000/50000, Loss:   1.4123
Iteration:  22000/50000, Loss:   1.4419
Iteration:  23000/50000, Loss:   1.4023
Iteration:  24000/50000, Loss:   1.3990
Iteration:  25000/50000, Loss:   1.3612


In [21]:
def sample_next(model, x, prev_state, topk=5, uniform=True):
    # Perform forward-prop and get the output of the last time-step
    out, state = model(x, prev_state)
    last_out = out[0, -1, :]

    # Get the top-k indexes and their values
    topk = topk if topk else last_out.shape[0]
    top_logit, top_ix = torch.topk(last_out, k=topk, dim=-1)
    
    # Get the softmax of the topk's and sample
    p = None if uniform else F.softmax(top_logit.detach(), dim=-1).numpy()
    sampled_ix = np.random.choice(top_ix, p=p)
    return sampled_ix, state


def sample(model, seed, topk=5, uniform=True, max_seqlen=18, stop_on=None):
    seed = seed if isinstance(seed, (list, tuple)) else [seed]
    
    model.eval()
    with torch.no_grad():
        sampled_ix_list = seed[:]
        x = torch.tensor([seed])
        
        prev_state = model.init_state(b_size=1)
        for t in range(max_seqlen - len(seed)):
            sampled_ix, prev_state = sample_next(model, x, prev_state, topk, uniform)

            sampled_ix_list.append(sampled_ix)
            x = torch.tensor([[sampled_ix]])
            
            if sampled_ix==stop_on:
                break
    
    model.train()
    return sampled_ix_list

In [28]:
print(">>> Samples where seed is a randomly chosen character.")
for i in range(10):
    seed = random.choice(list(char_to_ix.values())[1:])
    print(seed, "=>", " ".join(keys_to_values(
        sample(model, seed, 5, False, 30, char_to_ix["<EOS>"]),
        ix_to_char, "<?>")))


print(">>> Samples where seed is a list of character.")
for i in range(3):
    seed = keys_to_values(list("python"), char_to_ix, char_to_ix["<PAD>"])
    print(seed, "=>", "".join(keys_to_values(
        sample(model, seed, 5, False, 30, char_to_ix["<EOS>"]),
        ix_to_char, "<?>")))

>>> Samples where seed is a randomly chosen character.
5 => e o m a s a u r u s <EOS>
24 => x i a n z u s a u r u s <EOS>
7 => g u l u a n <EOS>
14 => n e o d r o m u s <EOS>
5 => e u r o p e l t a <EOS>
13 => m o n g o s a u r u s <EOS>
5 => e u l o r n i t h o u s <EOS>
11 => k u n g i n i n s a u r u s <EOS>
16 => p o r a c h i o s a u r u s <EOS>
19 => s a u r o s a u r u s <EOS>
>>> Samples where seed is a list of character.


KeyError: '<PAD>'